In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, Concatenate
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
import os

# Load the dataset
data = pd.read_csv("dataset.csv")

# Extract the image paths and additional features
image_paths = data["image_path"]
labels = data["artist_label"]
additional_features = data.drop(columns=["image_path", "artist_label"])

num_artists = len(os.listdir('dataset/'))

# Function to load and preprocess images
def load_and_preprocess_images(image_paths, target_size=(128, 128)):
    images = []
    for img_path in image_paths:
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        images.append(img_array)
    return np.array(images)

# Load and preprocess images
X_images = load_and_preprocess_images(image_paths)
X_images = X_images / 255.0  # Normalize the images to [0, 1] range

# Convert labels to numpy array and one-hot encode them
y = np.array(labels)
y = to_categorical(y, num_classes=num_artists)

# Convert additional features to numpy array
X_additional = additional_features.to_numpy()

# Normalize the additional features using StandardScaler
scaler = StandardScaler()
X_additional = scaler.fit_transform(X_additional)

# Split data into training and testing sets
X_train_images, X_test_images, X_train_additional, X_test_additional, y_train, y_test = train_test_split(
    X_images, X_additional, y, test_size=0.2, random_state=42
)

# Load the pre-trained VGG16 model without the top layers (classification part)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model layers so that their weights are not updated during training
base_model.trainable = False

# Image input branch
image_input = Input(shape=(128, 128, 3), name='image_input')
x = base_model(image_input)
x = Flatten()(x)

# Additional features input branch
additional_input = Input(shape=(X_additional.shape[1],), name='additional_input')
y = Dense(64, activation='relu')(additional_input)

# Concatenate image features with the processed additional features
z = Concatenate()([x, y])

# Fully connected layers for classification
z = Dense(128, activation='relu')(z)
z = Dropout(0.5)(z)
output = Dense(num_artists, activation='softmax')(z)  # num_artists classes, softmax activation

# Define the complete model
model = Model(inputs=[image_input, additional_input], outputs=output)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Categorical cross-entropy for multi-class classification
              metrics=['accuracy'])

# Set up EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy',  # Monitor validation accuracy
                               patience=3,  # Stop after 3 epochs without improvement
                               restore_best_weights=True,  # Restore the best weights when stopping
                               verbose=1)

# Train the model with EarlyStopping
history = model.fit(
    [X_train_images, X_train_additional],  # Pass both image and additional features for training
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=([X_test_images, X_test_additional], y_test),  # Validation data
    callbacks=[early_stopping]  # Include EarlyStopping in the callbacks
)
# Evaluate the model
test_loss, test_acc = model.evaluate([X_test_images, X_test_additional], y_test)
print(f"Test accuracy: {test_acc}")

# Predict on the test set
y_pred = np.argmax(model.predict([X_test_images, X_test_additional]), axis=1)  # Convert softmax probabilities to class labels

# Get the actual class labels (as integers)
y_true = np.argmax(y_test, axis=1)


c:\Users\zq\Documents\GitHub\yukien\venv\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 59s 838ms/step - accuracy: 0.5958 - loss: 1.1884 - val_accuracy: 0.8554 - val_loss: 0.4433
Epoch 2/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 63s 903ms/step - accuracy: 0.8299 - loss: 0.4943 - val_accuracy: 0.8893 - val_loss: 0.3334
Epoch 3/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 66s 942ms/step - accuracy: 0.8738 - loss: 0.3518 - val_accuracy: 0.9071 - val_loss: 0.2851
Epoch 4/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 65s 931ms/step - accuracy: 0.9152 - loss: 0.2633 - val_accuracy: 0.9196 - val_loss: 0.2496
Epoch 5/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 63s 907ms/step - accuracy: 0.9299 - loss: 0.2243 - val_accuracy: 0.8982 - val_loss: 0.2817
Epoch 6/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 63s 902ms/step - accuracy: 0.9330 - loss: 0.1977 - val_accuracy: 0.9321 - val_loss: 0.2274
Epoch 7/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 63s 908ms/step - accuracy: 0.9449 - loss: 0.1567 - val_accuracy: 0.9304 - val_loss: 0.2448
Epoch 8/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 62s 892ms/step - accuracy: 0.9663 - loss: 0.1192 - val_accu

In [4]:

# Classification report for performance evaluation
report = classification_report(y_true, y_pred, 
                              target_names=[f'Class {i}' for i in np.unique(y_true)])
print(report)



              precision    recall  f1-score   support

     Class 0       0.95      0.89      0.92       177
     Class 1       0.97      0.93      0.95        42
     Class 2       0.92      0.98      0.95       271
     Class 3       0.89      0.84      0.86        38
     Class 4       0.93      0.88      0.90        32

    accuracy                           0.93       560
   macro avg       0.93      0.90      0.92       560
weighted avg       0.93      0.93      0.93       560



precision    recall  f1-score   support

     Class 0       0.93      0.93      0.93       180
     Class 1       0.97      0.91      0.94        43
     Class 2       0.96      0.97      0.96       264
     Class 3       0.91      0.91      0.91        43

    accuracy                           0.94       530
   macro avg       0.94      0.93      0.93       530
weighted avg       0.94      0.94      0.94       530

              precision    recall  f1-score   support

     Class 0       0.95      0.89      0.92       177
     Class 1       0.97      0.93      0.95        42
     Class 2       0.92      0.98      0.95       271
     Class 3       0.89      0.84      0.86        38
     Class 4       0.93      0.88      0.90        32

    accuracy                           0.93       560
   macro avg       0.93      0.90      0.92       560
weighted avg       0.93      0.93      0.93       560